In [ ]:
#file to import the data
import numpy as np
import pandas as pd
import matplotlib as plt

#data read in: #now that the data files are in github this can be changed.
aqi_data = "/Users/catejeschke/Northwestern/EARTH361/Final Project Work/annual_aqi_by_county_2024.csv"
pop_data = "/Users/catejeschke/Northwestern/EARTH361/Final Project Work/Population by Race and Ethnicity for Illinois and its Counties.csv"
vehicle_counts = [79043, 161353, 20160, 2168964, 941343, 55432, 11059, 26460, 28854, 479476, 638988, 101371, 61751, 268498, 328479, 165130, 169226, 37157, 131702, 246376, 203862, 674131, 259060]
pd_aqi = pd.read_csv(aqi_data)
population_data = pd.read_csv(pop_data)

#this data is limited to specific counties
#print(pd_aqi)

illinois_pd = pd_aqi[pd_aqi["State"] == "Illinois"]
#print(illinois_pd)
#now select wanted columns: Median AQI, Good Days, Days moderate to unhealthy, Days  CO - PM 10
illinois_pd["Moderate to Unhealthy Days"] = illinois_pd["Moderate Days"] + illinois_pd["Unhealthy Days"] + illinois_pd["Very Unhealthy Days"] + illinois_pd["Hazardous Days"] + illinois_pd["Unhealthy for Sensitive Groups Days"]

#print(illinois_pd.columns)


illinois_pd = illinois_pd.drop(['Year', 'Moderate Days', 'Unhealthy for Sensitive Groups Days',
       'Unhealthy Days', 'Very Unhealthy Days', 'Hazardous Days', 'Max AQI',
       '90th Percentile AQI'], axis=1)

#print(illinois_pd["County"])
illinois_pd["Vehicle Counts"] = vehicle_counts
#print(illinois_pd["Vehicle Counts"])
#print(population_data.head())
population_data["Name"] = population_data["Name"].str.replace("County", "")
population_data = population_data[["Name","Total Population"]]
population_data = population_data.rename(columns={"Name": "County"})
illinois_pd["County"] = illinois_pd["County"].str.replace("County", "").str.strip()
population_data["County"] = population_data["County"].str.replace("County", "").str.strip()

population_filtered = population_data[ population_data["County"].isin(illinois_pd["County"]) ]
new_attempt = pd.merge(illinois_pd, population_filtered, how="left", on="County")

print(new_attempt.head())




      State     County  Days with AQI  Good Days  Median AQI  Days CO  \
0  Illinois      Adams            233        210          42        0   
1  Illinois  Champaign            366        250          44        0   
2  Illinois      Clark            363        332          36        0   
3  Illinois       Cook            366        131          54        0   
4  Illinois     DuPage            366        230          45        0   

   Days NO2  Days Ozone  Days PM2.5  Days PM10  Moderate to Unhealthy Days  \
0         0         233           0          0                          23   
1         0         194         172          0                         116   
2         0         363           0          0                          31   
3        20         123         208         15                         235   
4         0         134         232          0                         136   

   Vehicle Counts Total Population  
0           79043           65,737  
1          161353 

/var/folders/x8/tqp39nqn2bl3tq0708_4d25m0000gn/T/ipykernel_89877/2739997116.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  illinois_pd["Moderate to Unhealthy Days"] = illinois_pd["Moderate Days"] + illinois_pd["Unhealthy Days"] + illinois_pd["Very Unhealthy Days"] + illinois_pd["Hazardous Days"] + illinois_pd["Unhealthy for Sensitive Groups Days"]


In [ ]:
#for the AFS data
import numpy as np
import pandas as pd
import matplotlib as plt

afs_data = "Alternative_Fueling_Stations_5630652685965129335.csv"
pd_afs = pd.read_csv(afs_data)

zip_county = "Zip Code to County Spreadsheet export 2025-12-02 22-31-09.zip"
zip = pd.read_csv(zip_county)

columns_to_drop = [
    "cards_accepted",
    "date_last_confirmed",
    "expected_date",
    "groups_with_access_code",
    "open_date",
    "owner_type_code",
    "related_stations",
    "restricted_access",
    "funding_sources",
    "station_phone",
    "updated_at",
    "country",
    "bd_blends",
    "cng_dispenser_num",
    "cng_fill_type_code",
    "cng_has_rng",
    "cng_psi",
    "cng_renewable_source",
    "cng_total_compression",
    "cng_total_storage",
    "cng_vehicle_class",
    "e85_blender_pump",
    "e85_other_ethanol_blends",
    "ev_dc_fast_num",
    "ev_level1_evse_num",
    "ev_level2_evse_num",
    "ev_connector_types",
    "ev_network",
    "ev_network_web",
    "ev_other_evse",
    "hy_is_retail",
    "hy_pressures",
    "hy_standards",
    "hy_status_link",
    "lng_has_rng",
    "lng_renewable_source",
    "lng_vehicle_class",
    "lpg_nozzle_types",
    "lpg_primary",
    "ng_fill_type_code",
    "ng_psi",
    "ng_vehicle_class",
    "rd_blended_with_biodiesel",
    "rd_blends",
    "rd_max_biodiesel_level",
    "nps_unit_name",
    "access_days_time_fr",
    "groups_with_access_code_fr",
    "federal_agency_id",
    "federal_agency_code",
    "federal_agency_name",
    "ev_network_ids_station",
    "ev_network_ids_posts",
    "intersection_directions",
    "access_detail_code",
    "plus4",
    "ev_renewable_source",
    "ev_workplace_charging",
    "maximum_vehicle_class",
    "rd_blends_fr",
    "intersection_directions_fr",
    "bd_blends_fr",
    "ev_pricing_fr",
    "ev_charging_units"
]

pd_afs = pd_afs.drop(columns=columns_to_drop)
pd_afs.head()

pd_afs = pd_afs[pd_afs["fuel_type_code"].isin(["ELEC"])]
#print(f"Filtered AFS shape:{pd_afs.shape} and value counts: {pd_afs["fuel_type_code"].value_counts()}")

pd_afs["zip"] = (
    pd_afs["zip"]
    .astype(str)
    .str.replace(r"\.0$", "", regex=True)
    .str.zfill(5)
)

unique_zips = pd_afs['zip'].drop_duplicates().reset_index(drop=True)
print(unique_zips)

#zip code data 
zip_il = zip[zip['USPS_ZIP_PREF_STATE'] == 'IL']
il_zip_county_df = zip_il.copy()

counties_aqi = [
    "Adams", "Champaign", "Clark", "Cook", "DuPage", "Effingham", "Hamilton",
    "Jersey", "Jo Daviess", "Kane", "Lake", "Macon", "Macoupin", "Madison",
    "McHenry", "McLean", "Peoria", "Randolph", "Rock Island", "Saint Clair",
    "Sangamon", "Will", "Winnebago"
]

filtered_df = il_zip_county_df[il_zip_county_df['County Clean'].isin(counties_aqi)]
unique_zips_df = filtered_df[['ZIP']].drop_duplicates().reset_index(drop=True)

set_of_afs_zips = set(unique_zips[unique_zips != '00nan'].astype(str))
set_of_aqi_zips = set(unique_zips_df['ZIP'].astype(str))
shared_unique_zips = list(set_of_afs_zips.intersection(set_of_aqi_zips))

pd_afs_filtered = pd_afs[pd_afs['zip'].isin(shared_unique_zips)]

print(f"Here are my columns: {pd_afs_filtered.columns}")

columns_to_remove = ['x', 'y']
pd_afs_filtered = pd_afs_filtered.drop(columns=columns_to_remove)

display(pd_afs_filtered.head())
pd_afs_filtered = pd_afs_filtered[pd_afs_filtered["fuel_type_code"].isin(["ELEC"])]

zip_count = (
    pd_afs_filtered.groupby(["zip", "fuel_type_code"])
       .size()
       .reset_index(name="count")
)

zip_to_county_mapping = il_zip_county_df[['ZIP', 'County Clean']].drop_duplicates(subset=['ZIP']).copy()
zip_to_county_mapping.rename(columns={'ZIP': 'zip', 'County Clean': 'County'}, inplace=True)

pd_afs_filtered['zip'] = pd_afs_filtered['zip'].astype(str)
zip_to_county_mapping['zip'] = zip_to_county_mapping['zip'].astype(str)

pd_afs_filtered = pd.merge(
    pd_afs_filtered,
    zip_to_county_mapping,
    on='zip',
    how='left'
)

print("DataFrame after adding county information:")
display(pd_afs_filtered.head())
#count for checking
county_count = (
    pd_afs_filtered.groupby(["County_x", "fuel_type_code"])
       .size()
       .reset_index(name="count")
)


0      61820
1      61761
2      60666
3      60176
4      61201
       ...  
382    62207
383    60471
384    60141
385    62305
386    00nan
Name: zip, Length: 387, dtype: object
Here are my columns: Index(['OBJECTID', 'access_code', 'access_days_time', 'fuel_type_code', 'id',
       'status_code', 'facility_type', 'station_name', 'geocode_status',
       'latitude', 'longitude', 'city', 'state', 'street_address', 'zip',
       'ev_pricing', 'x', 'y'],
      dtype='object')


,OBJECTID,access_code,access_days_time,fuel_type_code,id,status_code,facility_type,station_name,geocode_status,latitude,longitude,city,state,street_address,zip,ev_pricing
72,2023,public,24 hours daily; pay lot,ELEC,33921,E,PAY_GARAGE,City of Champaign - Hill Street Parking Deck,GPS,40.119161,-88.244185,Champaign,IL,123 W Hill St,61820,Free
95,2754,public,24 hours daily,ELEC,38855,E,MUNI_GOV,City Hall Annex - Town of Normal,GPS,40.507796,-88.982897,Normal,IL,100 E Phoenix Ave,61761,Free
96,2755,public,24 hours daily; parking fee,ELEC,38856,E,PARKING_GARAGE,College Avenue Parking Deck,GPS,40.511172,-88.984985,Normal,IL,101 W Mulberry St,61761,Free
104,3137,public,24 hours daily; pay lots,ELEC,40794,E,AIRPORT,O'Hare Airport,GPS,41.978506,-87.903280,Chicago,IL,10000 W O'Hare Ave,60666,Free
110,3288,private,NaN,ELEC,42460,E,FIRE_STATION,Town of Normal - Fire Department Headquarters,200-9,40.505132,-88.994211,Normal,IL,606 S Main St,61761,NaN


DataFrame after adding county information:


,OBJECTID,access_code,access_days_time,fuel_type_code,id,status_code,facility_type,station_name,geocode_status,latitude,longitude,city,state,street_address,zip,ev_pricing,County
0,2023,public,24 hours daily; pay lot,ELEC,33921,E,PAY_GARAGE,City of Champaign - Hill Street Parking Deck,GPS,40.119161,-88.244185,Champaign,IL,123 W Hill St,61820,Free,Champaign
1,2754,public,24 hours daily,ELEC,38855,E,MUNI_GOV,City Hall Annex - Town of Normal,GPS,40.507796,-88.982897,Normal,IL,100 E Phoenix Ave,61761,Free,McLean
2,2755,public,24 hours daily; parking fee,ELEC,38856,E,PARKING_GARAGE,College Avenue Parking Deck,GPS,40.511172,-88.984985,Normal,IL,101 W Mulberry St,61761,Free,McLean
3,3137,public,24 hours daily; pay lots,ELEC,40794,E,AIRPORT,O'Hare Airport,GPS,41.978506,-87.903280,Chicago,IL,10000 W O'Hare Ave,60666,Free,Cook
4,3288,private,NaN,ELEC,42460,E,FIRE_STATION,Town of Normal - Fire Department Headquarters,200-9,40.505132,-88.994211,Normal,IL,606 S Main St,61761,NaN,McLean


In [ ]:
"""
import plotly.graph_objects as go

pd_afs_filtered["hover_text"] = (
    pd_afs_filtered["station_name"]
    + "<br>County: " + pd_afs_filtered["County"]
    + "<br>ZIP: " + pd_afs_filtered["zip"]
)

fig = go.Figure(
    data=go.Scattergeo(
        lon = pd_afs_filtered['longitude'],
        lat = pd_afs_filtered['latitude'],
        text = pd_afs_filtered['hover_text'],
        mode = 'markers',
        marker = dict(
            size = 6,
            color = "blue",
            opacity = 0.8
        )
    ))

fig.update_layout(
    title = 'Alternative Fuel Station Distribution in Illinois',
    geo_scope='usa',
    geo = dict(
        center = dict(lat=40.0, lon=-89.0),
        lataxis_range=[36.5, 42.8],
        lonaxis_range=[-98.0, -80.0],
        scope='usa'
    )
)
fig.show()
"""

ModuleNotFoundError: No module named 'plotly'